In [1]:
# Import Taichi
import taichi as ti
import taichi.math as tm
import time

[Taichi] version 1.7.0, llvm 15.0.1, commit 2fd24490, win, python 3.10.6


In [2]:
ti.init(arch=ti.vulkan)

# type aliases
vec2 = tm.vec2
vec3 = tm.vec3

[Taichi] Starting on arch=vulkan


In [3]:
# define resolution
WIDTH = 800
HEIGHT = 800

# configurations
PARTICLE_COUNT = 100 # define shape
PARTICLE_RADIUS = 0.01 # for rendering only
USE_GRAVITY = False
GRAVITY = ti.Vector([0.0, 0.01, 0.0])

# for particle simulation only
HALF_PARTICLE_COUNT = PARTICLE_COUNT // 2

# for cloth simulation only
GIRD_SIZE = vec2(10, 10)
GAP = vec2(1 / GRID_SIZE.x, 1 / GRID_SIZE.y)
STRUCTURE_CONNECTION_COUNT = 100
CONNECTION_COUNT = STRUCTURE_CONNECTION_COUNT
STIFFNESS = [0.3, 1, 1]

# define variables - general
masses = ti.field(dtype=float, shape=PARTICLE_COUNT)
positions = ti.Vector.field(3, dtype=float, shape=PARTICLE_COUNT)
previous_positions = ti.Vector.field(3, dtype=float, shape=PARTICLE_COUNT)

# variables - springs
rest_lengths = ti.field(ti.f32, shape=HALF_PARTICLE_COUNT)
connection_starts = ti.field(ti.i32, shape=HALF_PARTICLE_COUNT)
connection_ends = ti.field(ti.i32, shape=HALF_PARTICLE_COUNT)
stiffness_amounts = ti.field(ti.f32, shape=HALF_PARTICLE_COUNT)

In [4]:
# Utility Functions
@ti.func
def generate_random_float(minInclude, maxInclude):
    random_float = minInclude + (maxInclude - minInclude) * ti.random()
    return random_float

@ti.func
def random_position() -> vec3:
    x = generate_random_float(0.5, 1)
    y = generate_random_float(0.5, 1)
    return vec3(x, y, 0)

@ti.func
def random_velocity(minInclude: vec2, maxInclude: vec2) -> vec3:
    x = generate_random_float(minInclude.x, maxInclude.x)
    y = generate_random_float(minInclude.y, maxInclude.y)
    return vec3(x, y, 0)

@ti.func
def grid_position(row: int, col: int) -> vec3:
    x = 0 + col * GAP.x
    y = 0 + row * GAP.y
    return vec3(x, y, 0)

@ti.func
def structure_connection():
    pass

In [5]:
@ti.func
def init_connections():
    for i in range(HALF_PARTICLE_COUNT):
        connection_starts[i] = i * 2
        connection_ends[i] = i * 2 + 1

@ti.func
def init_connections_cloth():
    pass
        
@ti.func
def init_rest_lengths():
    for i in range(HALF_PARTICLE_COUNT):
        start_index = connection_starts[i]
        end_index = connection_ends[i]
    
        particle_start = positions[start_index]
        particle_end = positions[end_index]
        vector = particle_end - particle_start
        magnitude = vector.norm()

        rest_lengths[i] = magnitude

@ti.kernel
def init():
    # particle simulations
    # for i in range(PARTICLE_COUNT):
    #     positions[i] = random_position()
    #     previous_positions[i] = positions[i] - random_velocity(vec2(-1), vec2(1))
    # init_connections()
    # init_rest_lengths()
    # cloth simulations
    init_connections_cloth()
    init_rest_lengths()

In [6]:
@ti.kernel
def update(delta_time: float):
    for i in range(PARTICLE_COUNT):
        current_position = positions[i]
        # verlet simulation
        next_position = 2.0 * current_position - previous_positions[i]
        if USE_GRAVITY: next_position -= GRAVITY
        
        positions[i] = next_position
        previous_positions[i] = current_position

In [7]:
@ti.func
def apply_shape_constraints():
    for i in range(HALF_PARTICLE_COUNT):
        # gain indicies
        start_index = connection_starts[i]
        end_index = connection_ends[i]
        # gain particle position value
        particle_start = positions[start_index]
        particle_end = positions[end_index]
        # calculate offset
        vector = particle_start - particle_end
        current_length = vector.norm()
        difference = rest_lengths[i] - current_length
        percentage = difference / current_length / 2
        offset = vector * percentage * STIFFNESS[0]
        # apply to real value
        positions[start_index] += offset * 0.5
        positions[end_index] -= offset * 0.5

@ti.func
def apply_boundary_constranits():
    for i in range(PARTICLE_COUNT):
         # boundary constraints for x-axis
        if positions[i].x > 1:
            previous_positions[i].x = positions[i].x
            positions[i].x = 1
        elif positions[i].x < 0:
            previous_positions[i].x = positions[i].x 
            positions[i].x = 0

        # boundary constraints for y-axis
        if positions[i].y > 1:
            previous_positions[i].y = positions[i].y
            positions[i].y = 1
        elif positions[i].y < 0:
            previous_positions[i].y = positions[i].y
            positions[i].y = 0

@ti.kernel
def apply_constriants():
    apply_shape_constraints()
    apply_boundary_constranits()

In [8]:
init()

window = ti.ui.Window("Mass Spring System", (WIDTH, HEIGHT), vsync=True)
# canvas to render a scene
canvas = window.get_canvas()
canvas.set_background_color((1, 1, 1))

# Setting up the camera
scene = window.get_scene()
camera = ti.ui.Camera()
camera.position(0.5, 0.5, 1.25)
camera.lookat(0.5, 0.5, 0) 
camera.up(0, 1, 0) 
scene.set_camera(camera)
scene.ambient_light(color=(1, 1, 1))

gui = window.get_gui()

boundaries = ti.Vector.field(3, dtype=float, shape=(4, ))
boundaries[0] = [0, 0, 0]
boundaries[1] = [1, 0, 0]
boundaries[2] = [0, 1, 0]
boundaries[3] = [1, 1, 0]

static_sphere = ti.Vector.field(3, dtype=float, shape=(1, ))
static_sphere[0] = [0, 0, 0]

previous_time = time.time()

RESTART_CLICKED = False

while window.running:
    # update delta time
    current_time = time.time()
    delta_time = current_time - previous_time
    previous_time = current_time

    if(RESTART_CLICKED): init()
    
    # simulation update
    update(delta_time)
    apply_constriants()
    
    # simulation rendering
    scene.particles(positions, radius=PARTICLE_RADIUS, color=(0.25, 0.25, 0.25))
    scene.particles(boundaries, radius=PARTICLE_RADIUS, color=(0, 1, 0))
    scene.particles(static_sphere, radius=PARTICLE_RADIUS, color=(1, 0, 0))
    
    scene.lines(positions, color = (0.28, 0.68, 0.99), width = 2.0)
    
    with gui.sub_window("Configurations", x=0.02, y=0.02, width=0.3, height=0.25):
        RESTART_CLICKED = gui.button("Restart")
        USE_GRAVITY = gui.checkbox("Enable Gravity", USE_GRAVITY)
        gui.text("Stiffness")
        STIFFNESS[0] = gui.slider_float("Sturcture", STIFFNESS[0], minimum=0, maximum=1)
        STIFFNESS[0] = gui.slider_float("Shear", STIFFNESS[0], minimum=0, maximum=1)
        STIFFNESS[0] = gui.slider_float("Flexion", STIFFNESS[0], minimum=0, maximum=1)

    # flush
    canvas.scene(scene)
    window.show()